In [2]:
import os
from mlflow.tracking import MlflowClient

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 1.26.1


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset. 

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

In [6]:
!python ./utils/preprocess_data.py --raw_data_path ./data --dest_path ./output
os.listdir('./output')

['dv.pkl', 'test.pkl', 'train.pkl', 'valid.pkl']

## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

In [10]:
# !mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

^C


http://localhost:5000/

In [12]:
!python ./utils/train.py

2022/05/31 09:12:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/31 09:12:30 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.
2022/05/31 09:15:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ognen\miniconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [26]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.get_run('3e5c8f10dce3456a9f38b8cf8a9f156c').data.to_dictionary()['params']

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '10',
 'max_features': 'auto',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '0',
 'verbose': '0',
 'warm_start': 'False'}

## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

In [3]:
# !mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

In [5]:
!python ./utils/hpo.py


100%|██████████| 50/50 [55:03<00:00, 66.06s/trial, best loss: 6.6284257482044735]


http://localhost:5000/

In [7]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>]

In [14]:
runs = client.search_runs(
    experiment_ids='2',
    max_results=3,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.3f}")

run id: 62fd3e8f6241437abf7de9c4d015b7fa, rmse: 6.628
run id: cee2d27cd5bb47bcb21225ca693ab440, rmse: 6.630
run id: 8f4c3e00975243ab995fb667a71d4e1d, rmse: 6.630


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

In [1]:
!python ./utils/register_model.py

2022/06/02 21:02:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ognen\miniconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forest-best-models'.
2022/06/02 21:10:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best-models, version 1
Created version '1' of model 'random-forest-best-models'.


In [3]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [22]:
client.get_registered_model("random-forest-best-models")._latest_version

[<ModelVersion: creation_timestamp=1654168250788, current_stage='None', description='', last_updated_timestamp=1654168250788, name='random-forest-best-models', run_id='b6ee90c8ffc54d2498956757409cdd5b', run_link='', source='./mlruns/3/b6ee90c8ffc54d2498956757409cdd5b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>]

In [17]:
client.get_run(run_id='b6ee90c8ffc54d2498956757409cdd5b')

<Run: data=<RunData: metrics={'test_rmse': 6.547886185595423,
 'training_mae': 3.7900404294512096,
 'training_mse': 31.131050948533428,
 'training_r2_score': 0.7671660211119171,
 'training_rmse': 5.579520673725784,
 'training_score': 0.7671660211119171,
 'valid_rmse': 6.6284257482044735}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': '19',
 'max_features': 'auto',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '3',
 'min_samples_split': '5',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '28',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '42',
 'verbose': '0',
 'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
 'estimator_name': 'RandomForestRegressor',
 'mlflow.log-model.history': '[{"run_id": "b6ee90c8ffc54d2498956757409cdd5b", '
                             '"artifact_path": "model", "utc_time_created": '
    

In [20]:
client.get_metric_history(run_id='b6ee90c8ffc54d2498956757409cdd5b', key='test_rmse')

[<Metric: key='test_rmse', step=0, timestamp=1654167743061, value=6.547886185595423>]